# Opening A New Sushi Chain in San Francisco, California, USA
## The Battle of the Neighborhoods 1
### Coursera Applied Data Science Capstone by IBM

In [21]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Table of contents
* [Business Problem Introduction](#businessproblemintroduction)
* [Data Methodology](#datamethodology)

## Business Problem Introduction <a name="businessproblemintroduction"></a>

In this project we will try to find a suitable location for a Japanese chain who will be opening a new brick and mortar restaurant in a new geographic market. The key stakeholder will be the Director of Operations who is interested in opening a *sushi restaurant* in *San Francisco, California, USA*. 

The Director has specified two criteria: first, as San Francisco is an international destination with many eateries, the Director has specifically requested a location *not crowded with competitor cuisines*. Second, since the raw materials and ingredients for maintaining a restaurant of the chain's calibre are expensive, the Director would also prefer a city location *with low theft rates*. Since the chain maintains an active Loss Prevention program with specialized equipment, a location with low crime theft over the past 5 years is acceptable.

We will be analyzing locations in San Francisco as well as crime data to support the restaurant chain's initiative. We will then present pros and cons of areas selected and provide recommendations to the stakeholders.

## Data Methodology <a name="datamethodology"></a>

Based on the requirements defined in the problem, factors that will influence the stakeholder's decision are:
* number of competitor restaurants in a location
* crime data for a location

For the first criteria, we plan to use data from Foursquare to generate a list of eateries in San Francisco by location. For the second criteria, we will use the public San Francisco crimes data to investigate theft and robberies by location to present a proposal for site selection.

We decided to use key district parameters to define San Francisco locations since San Francisco is no larger than approximately 7 miles by 7 miles. Data sources initially planned for extraction and generation include:
* number of eateries and their type by San Francisco location will be obtained using **Foursquare API**
* theft locations of the city will be used and overlayed onto a San Franciso **Folium map** for analysis and analysis

## Analysis (coming soon) <a name="analysis"></a>

In [18]:
#import requests
#import urllib.request
#import pandas as pd
#import csv
#from bs4 import BeautifulSoup
#!conda install -c conda-forge folium=0.5.0 --yes 
#import folium # map rendering library
#print('Libraries imported.')

In [19]:
#Foursquare info
#CLIENT_ID = '1H41W44OYMI2XZ5Y13TAWB0YDLCMITZJSXCLELDNFRGLB23G' # your Foursquare ID
#CLIENT_SECRET = 'TMYVWORZW4OEW1NCQUQWYNX2DKF2X2D340ILPIZLKCGO5DVF' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version


In [20]:
#url='#########'
#df=pd.read_html(url, header=0)[0]
#df.head()

## Results and Discussion (coming soon) <a name="resultsdiscussion"></a>

## Conclusion (coming soon) <a name="conclusion"></a>